In [1]:
import torch

from transformers import SamModel, SamProcessor
from diffusers import AutoPipelineForInpainting

import app_new

### Download Models

In [2]:
sam_model_name = "facebook/sam-vit-base"
diffusion_model_name = "diffusers/stable-diffusion-xl-1.0-inpainting-0.1"


device = "cuda" if torch.cuda.is_available() else "cpu"
model = SamModel.from_pretrained(sam_model_name).to(device)
processor = SamProcessor.from_pretrained(sam_model_name)

pipeline = AutoPipelineForInpainting.from_pretrained(diffusion_model_name, torch_dtype=torch.float16)
# This will make it more efficient on our hardware
pipeline.enable_model_cpu_offload()

Loading pipeline components...:   0%|          | 0/7 [00:00<?, ?it/s]

The config attributes {'decay': 0.9999, 'inv_gamma': 1.0, 'min_decay': 0.0, 'optimization_step': 37000, 'power': 0.6666666666666666, 'update_after_step': 0, 'use_ema_warmup': False} were passed to UNet2DConditionModel, but are not expected and will be ignored. Please verify your config.json configuration file.


### Interactive App

In [ ]:
# my_app = app_new.generate_app(get_processed_inputs, inpaint)

my_app = app_new.generate_app(processor, model, pipeline, device)

Running on local URL:  http://127.0.0.1:7860
IMPORTANT: You are using gradio version 3.50.2, however version 4.44.1 is available, please upgrade.
--------
Running on public URL: https://20cf3f108e0853db35.gradio.live

This share link expires in 72 hours. For free permanent hosting and GPU upgrades, run `gradio deploy` from Terminal to deploy to Spaces (https://huggingface.co/spaces)


(1006, '')


c:\Users\Windows\AppData\Local\Programs\Python\Python312\Lib\site-packages\transformers\models\clip\modeling_clip.py:480: UserWarning: 1Torch was not compiled with flash attention. (Triggered internally at C:\actions-runner\_work\pytorch\pytorch\builder\windows\pytorch\aten\src\ATen\native\transformers\cuda\sdp_utils.cpp:555.)
  attn_output = torch.nn.functional.scaled_dot_product_attention(


  0%|          | 0/49 [00:00<?, ?it/s]

Keyboard interruption in main thread... closing server.


In [1]:
my_app.close()

NameError: name 'my_app' is not defined